In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
from ipywidgets import FloatProgress, Layout
from IPython.display import display
import micasense.imageset as imageset
import micasense.capture as capture
import os, glob
import multiprocessing

panelNames = None
useDLS = True

imagePath = './data/input/captures'
panelPath = './data//input/panel'
panelNames = glob.glob(os.path.join(panelPath,'IMG_0000_*.tif'))

outputPath = './data//output/stacks'
if not os.path.exists(outputPath):
    os.makedirs(outputPath)
thumbnailPath = './data//output/rgb'
if not os.path.exists(thumbnailPath):
    os.makedirs(thumbnailPath)

overwrite = False # Set to False to continue interrupted processing
generateThumbnails = True

# Allow this code to align both radiance and reflectance images; bu excluding
# a definition for panelNames above, radiance images will be used
# For panel images, efforts will be made to automatically extract the panel information
# but if the panel/firmware is before Altum 1.3.5, RedEdge 5.1.7 the panel reflectance
# will need to be set in the panel_reflectance_by_band variable.
# Note: radiance images will not be used to properly create NDVI/NDRE images below.
if panelNames is not None:
    panelCap = capture.Capture.from_filelist(panelNames)
else:
    panelCap = None

if panelCap is not None:
    if panelCap.panel_albedo() is not None:
        panel_reflectance_by_band = panelCap.panel_albedo()
    else:
        panel_reflectance_by_band = [0.65]*len(panelCap.images) #inexact, but quick
    panel_irradiance = panelCap.panel_irradiance(panel_reflectance_by_band)
    img_type = "reflectance"
else:
    if useDLS:
        img_type='reflectance'
    else:
        img_type = "radiance"


In [9]:
## This progress widget is used for display of the long-running process
f = FloatProgress(min=0, max=1, layout=Layout(width='100%'), description="Loading")
display(f)
def update_f(val):
    if (val - f.value) > 0.005 or val == 1: #reduces cpu usage from updating the progressbar by 10x
        f.value=val

%time imgset = imageset.ImageSet.from_directory(imagePath, progress_callback=update_f)
update_f(1.0)

FloatProgress(value=0.0, description='Loading', layout=Layout(width='100%'), max=1.0)

CPU times: user 491 ms, sys: 37.8 ms, total: 529 ms
Wall time: 2.66 s


In [10]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import micasense.imageutils as imageutils
import micasense.plotutils as plotutils

alignment_img = glob.glob(os.path.join(imagePath,'IMG_0036_*.tif'))
cpt = capture.Capture.from_filelist(alignment_img)

## Alignment settings
match_index = 4 # Index of the band, here we use green
max_alignment_iterations = 20
warp_mode = cv2.MOTION_HOMOGRAPHY # MOTION_HOMOGRAPHY or MOTION_AFFINE. For Altum images only use HOMOGRAPHY
pyramid_levels = 3 # for 10-band imagery we use a 3-level pyramid. In some cases

print("Alinging images. Depending on settings this can take from a few seconds to many minutes")
# Can potentially increase max_iterations for better results, but longer runtimes
warp_matrices, alignment_pairs = imageutils.align_capture(cpt,
                                                          ref_index = match_index,
                                                          max_iterations = max_alignment_iterations,
                                                          warp_mode = warp_mode,
                                                          pyramid_levels = pyramid_levels)

print("Finished Aligning, warp matrices={}".format(warp_matrices))

Alinging images. Depending on settings this can take from a few seconds to many minutes
Finished aligning band 4
Finished aligning band 2
Finished aligning band 3
Finished aligning band 1
Finished aligning band 6
Finished aligning band 8
Finished aligning band 9
Finished aligning band 7
Finished aligning band 0
Finished aligning band 5
Finished Aligning, warp matrices=[array([[ 1.0040864e+00, -7.4430788e-03, -3.3286095e+01],
       [ 7.0750872e-03,  1.0027938e+00, -7.7721471e-01],
       [ 1.1506276e-06, -1.8202343e-06,  1.0000000e+00]], dtype=float32), array([[ 1.0003500e+00, -2.1300591e-03, -2.2057552e+01],
       [ 1.0440277e-03,  1.0018458e+00, -5.8781284e-01],
       [-2.2985755e-06, -2.5595128e-07,  1.0000000e+00]], dtype=float32), array([[ 9.9591613e-01, -5.0186333e-03, -1.6835062e+01],
       [ 3.8949682e-03,  9.9798226e-01,  1.2554541e+01],
       [-2.5367942e-06,  1.2353189e-07,  1.0000000e+00]], dtype=float32), array([[ 1.0020077e+00, -8.0345459e-03, -1.1813504e+01],
       

In [12]:
import exiftool
import datetime

use_multi_process = True # set to False for single-process saving
overwrite_existing = False # skip existing files, set to True to overwrite

## This progress widget is used for display of the long-running process
f2 = FloatProgress(min=0, max=1, layout=Layout(width='100%'), description="Saving")
display(f2)
def update_f2(val):
    f2.value=val

if not os.path.exists(outputPath):
    os.makedirs(outputPath)
if generateThumbnails and not os.path.exists(thumbnailPath):
    os.makedirs(thumbnailPath)

# Save out geojson data so we can open the image capture locations in our GIS
#with open(os.path.join(outputPath,'imageSet.json'),'w') as f:
#    f.write(str(geojson_data))

# If we didn't provide a panel above, irradiance set to None will cause DLS data to be used
try:
    irradiance = panel_irradiance+[0]
except NameError:
    irradiance = None

start_time = datetime.datetime.now()

# Save all captures in the imageset as aligned stacks
imgset.save_stacks(warp_matrices,
                     outputPath,
                     thumbnailPath,
                     irradiance = irradiance,
                     multiprocess=use_multi_process, 
                     overwrite=overwrite_existing, 
                     progress_callback=update_f2)

end_time = datetime.datetime.now()
update_f2(1.0)

print("Saving time: {}".format(end_time-start_time))
print("Alignment+Saving rate: {:.2f} captures per second".format(float(len(imgset.captures))/float((end_time-start_time).total_seconds())))



FloatProgress(value=0.0, description='Saving', layout=Layout(width='100%'), max=1.0)

Saving time: 0:00:36.452745
Alignment+Saving rate: 0.93 captures per second


In [13]:
# Rename stacks and rgb files
for c in imgset.captures:
    for path, suffix in zip((outputPath, thumbnailPath), ('.tif', '_rgb.jpg')):
        old = f'{path}/{c.uuid}{suffix}'
        if os.path.exists(old):
            new = f'{path}/{c.images[0].meta.get_item("File:FileName")[:8]}{suffix}'
            os.rename(old, new)